My love of Flask is well known. It's a great microframework that puts you in control of the pieces needed in your webapp. It comes with templating, requests, responses, cookies, sessions, routing and... really, that's it. Routes can be built by decorating functions or by inheriting from `flask.views.View` or `flask.views.MethodView` depending on your needs.

Something I've explored in the past is building a simple RESTful service using Flask and SQLAlchemy. However, that blog post fell into the trap of "Look at all this code!" that happens to me regretfully often. D: However, I'd like revisit the idea and explore the topic a little more in depth.

I will forewarn, that this post is still pretty code heavy. If that's the sort of thing that makes your eyes glaze over, you may want to go to [/r/dogs](https://www.reddit.com/r/dogs) instead.

#Quotr: Inspirational Quotes for Web 2.0

Okay, lame. But it's not another TODO list, though not much better. But by the end, hopefully we'll feel comfortable with Flask, Flask-SQLAlchemy, Marshmallow (12/10 serializer library) and a little bit of Flask-Restful (though, it's going to be used as a base that we're going to modify).

This post was written with Python 3.4, but it should work with at least 2.7 and 3.3. I'm not sure about others and I'm too lazy to add the deadsnakes PPA right now. D:

You can follow along with vim or PyCharm or whatever. The full application is at the bottom of the post as well if you'd rather peak now.

##Step One: Packages

Instead of incrementally installing packages as we need them, I'm just going to give the whole list now (as a pip command even!):

    pip install --user -U flask-sqlalchemy flask-restful
    pip install --user -U marshmallow --pre

That'll install the current Flask build, the extensions, and everything these depend on. The reason for `--pre` on Marshmallow is to work off the latest and greatest build. Yeah, betas. Just trust me though.

Flask-Restful isn't necessary, but it provides quite a bit of boilerplate that we'd otherwise write ourselves. Plus, it's a great little package. Flask-Classy could also be used as well just creating our own implementation using `flask.views.MethodView`. However, for now, we'll focus on using Flask-Restful.

##Step Two: Hello World Sucks

I could show you:

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return 'Hello world'

if __name__ == '__main__':
    app.run(debug=True)

But that sucks. No one cares. CRUD apps with names that are missing vowels are the Web 2.0 hotness. So let's write one of those instead. And we're going to serve up thought provoking quotes through a JSON rest interface. Let's consider what our basic data will look like:

In [21]:
from collections import namedtuple

# basic quote object
# and we can represent many quotes with [Quote(person=..., quote=...) [,..]]
Quote = namedtuple('Quote', ['person', 'quote'])

# Let's prepopulate an example quote list
quotes = [Quote(p, q) for p, q in [
        ('Herbert West', 'I was busy pushing bodies around as you well know '
         'and what would a note say, Dan? "Cat dead, details later"?'),
        ('Jack Burton', "You remember what ol' Jack Burton always says at a time like that: "
         '"Have ya paid your dues, Jack?" "Yessir, the check is in the mail."'),
        ('Igor', 'Well, why isn\'t it "Froaderick Fronkensteen"?')
]]

Those are all great movies (Reanimator, Big Trouble in Little China, and Young Frankenstein) and if you've not seen one (or all) of them, you need to close this post (well...minimize it), go find them on Netflix, Amazon Prime, Hulu, HBOGO or whatever and watch them. After you're done, come back.

In [22]:
from IPython.display import Image
Image(url="http://i.imgur.com/9VeIbMZ.gif")

Now that you'rer back (or have turned on one of those movies in the background), let's see how we can take those Quote objects and output them as JSON. This is where we'll turn to Marshmallow:

In [29]:
from marshmallow import Schema
from marshmallow.fields import String

class QuoteSchema(Schema):
    person = String()
    quote = String()

print(QuoteSchema().dump(quotes[0]))

MarshalResult(data={'quote': 'I was busy pushing bodies around as you well know and what would a note say, Dan? "Cat dead, details later"?', 'person': 'Herbert West'}, errors={})


That's it! No complicated anything. Marshmallow is the best library I've found for writing JSON serializers and it's strength shows here even in a simple example. We simply declare which fields we want and what type they should end up as. Writing custom fields, nesting schemas and defining all sorts of complex option is super easy, but I'll save that for another time.

And getting this to Flask is really easy as well:

In [ ]:
from flask import jsonify
from random import choice

app = Flask(__name__)

@app.route('/quote')
def rand_quote():
    q = choice(quotes)
    data = QuoteSchema().dump(q).data
    return jsonify(data)

if __name__ == '__main__':
    app.run(debug=True)

And if that's all I wanted to show you, I'd be done. But I'm not. What if we wanted to add a route for displaying all three quotes at the same time? It's pretty easy to do, but first, let's modify our QuoteSchema to support namespacing.

Namespacing, sometimes also refered to as enveloping, in JSON is essentially a list of objects inside another object. It ends up looking like this:

In [24]:
{'quotes': [
    {'person': 'Herbert West', 'quote': '...'},
    {'person': 'Jack Burton', 'quote': '...'},
    {'person': 'Igor', 'quote': '...'}
]}

{'quotes': [{'person': 'Herbert West', 'quote': '...'},
  {'person': 'Jack Burton', 'quote': '...'},
  {'person': 'Igor', 'quote': '...'}]}

If we don't do this, we try jsonifying a list which is invalid JSON (as it's just a list at the top level and not an object). Adding this is, as you can imagine, really easy:

In [25]:
from marshmallow import post_dump, pprint

class QuoteSchema(Schema):
    class Meta:
        additional = ('person', 'quote')
    
    @post_dump(raw=True)
    def envelope(self, data, many):
        ns = 'quotes' if many else 'quote'
        return {ns: data}
    
pprint(QuoteSchema().dump(quotes, many=True).data)

{'quotes': [{'person': 'Herbert West',
             'quote': 'I was busy pushing bodies around as you well know '
                      'and what would a note say, Dan? "Cat dead, details '
                      'later"?'},
            {'person': 'Jack Burton',
             'quote': "You remember what ol' Jack Burton always says at a "
                      'time like that: "Have ya paid your dues, Jack?" '
                      '"Yessir, the check is in the mail."'},
            {'person': 'Igor',
             'quote': 'Well, why isn\'t it "Froaderick Fronkensteen"?'}]}


There's two things here:

* QuoteSchema.Meta: Marshmallow can pick up on most Python datatypes and automatically convert them instead of us declaring "This is a string", "This is a string, too", "That's an integer". Instead, we'll simply tell Marshmallow to look for these attributes and just serialize them.

* `@post_dump` My favorite thing: decorators. This one simply inserts a step before returning the dumped data to the caller. In this case, we're just adding a namespace and chainging its name depending on if we're returning one quote or many quotes. By saying `raw=True` we're asking Marshmallow for the actual data that's been dumped already.
    
With that in mind, let's build our "Return all the quotes" route:

In [ ]:
@app.route('/quote/all')
def all_quotes():
    qs = QuoteSchema().dump(quotes, many=True).data
    return jsonify(qs)

That's awesome. But consider the fact that we'll want to add *more* quotes from *more awesome* movies. If we keep up with this using functions as routes, we're gonna have to multiplex a bunch of logic into one thing and frankly, I'm not a big fan cramming a bunch of stuff into one function. But Python provides a way to cleanly multiplex: classes.

##Step Three: Resources and Posting

If you're following along, go ahead and just delete the `rand_quote` and `all_quotes` end points, we won't be needing them anymore.

And this is where Flask-Restful comes into play. Flask-Restful is a toolkit for developing RESTful APIs in Flask. It was developed by Twilio and they thought it was so awesome, they open sourced it. There's request parsing, extensible class based routing, marshalling (though not as nice as Marshmallow's) and a few other things. We're going to just use the class based routing here, but I highly recommend exploring the package more thoroughly. By default, Flask-Restful provides a JSON response maker. Meaning, we can simply return a dictionary and Flask-Restful will automatically turn it into JSON.

In [ ]:
from flask.ext.restful import Resource, Api # could also be from flask_restful import ...

api = Api()

class SingleQuote(Resource):
    def get(self):
        """Return a single random quote"""
        q = choice(quotes)
        data = QuoteSchema().dump(q).data
        return data
    
    def post(self):
        """Add a quote"""
        # uh...wut do here?
        return {'pass': True}
    
api.register_resource(SingleQuote, '/quote')

api.init_app(app)

To add a quote we're gonna have to need to:

* Accept some JSON
* Turn the JSON into a Python dict
* Turn that dict into a Quote object
* Put that object into the quotes list
* Probably return something indicating success or failure
    
Accepting the JSON and converting it to Python is easy. `flask.request` provides a `get_json` method that will either return parsed JSON or False (I'm not a fan of multiple return type, but whatever). However, turning that JSON into a Quote object is something that we need to implement ourselves. Luckily, Marshmallow can do this, and makes it easy as well! Back to the QuoteSchema class!

In [26]:
class QuoteSchema(Schema):
    class Meta:
        additional = ('person', 'quote')
    
    @post_dump(raw=True)
    def envelope(self, data, many):
        ns = 'quotes' if many else 'quote'
        return {ns: data}
    
    def make_object(self, data):
        assert 'person' in data and 'quote' in data, "Must provide person and quote"
        return Quote(person=data['person'], quote=data['quote'])

# load one
pprint(QuoteSchema().load({'person':'Ash', 'quote':"Klaatu Barada Nikto."}).data)
# load many
pprint(QuoteSchema().load([
        {'person':'Ash', 'quote':"Klaatu Barada Nikto."}, 
        {'person': 'Shaun', 'quote': "You've got red on you."}
], many=True).data)

Quote(person='Ash', quote='Klaatu Barada Nikto.')
[Quote(person='Ash', quote='Klaatu Barada Nikto.'),
 Quote(person='Shaun', quote="You've got red on you.")]


Putting this data in our quotes list is actually really easy:

In [ ]:
from flask import request

class SingleQuote(Resource):
    def get(self):
        """Return a single random quote"""
        q = choice(quotes)
        data = QuoteSchema().dump(q).data
        return data
    
    def post(self):
        """Add a quote"""
        json = request.get_json()
        if not json:
            return {'success': False, 'error': 'malformed request'} # probably should do something better
        
        else:
            try:
                if 'quotes' in json:
                    data = QuoteSchema().load(json['quotes'], many=True).data
                    quotes.extend(data)
                    num = len(data)
                else:
                    data = QuoteSchema().load(json['quote']).data
                    quotes.append(data)
                    num = 1
                return {'success': True, 'added': num}
            except KeyError as e: # super robust
                return {'success': False, 'error': repr(e)}

With this in place, you can fire up your favorite developer tool (I prefer Postman REST) and post some json like:

    {"quotes": [
        {"person": "Ash", "quote":" Klaatu Barada Nikto."}, 
        {"person": "Shaun", "quote": "You've got red on you."}
    ]}

Or:

    {"quote":
        {"person": "Ash", "quote":" Klaatu Barada Nikto."}
    }
    
to the application and them added to the quotes list! Just browse on over to http://localhost:5000/quotes.

...Oh, we haven't implemented a way to view *all* the quotes yet. Let's do that now, and while we're at it, let's add a way to view a specific quote:

In [ ]:
class AllQuotes(Resource):
    def get(self):
        return QuoteSchema(many=True).dump(quotes).data

class SingleQuote(Resource):
    def get(self, id=None):
        if id is None or not 0 <= id len(quotes):
            return choice(quotes)
        return quotes[id]

    def post(self):
        #same as above
        pass
    
api.register_resource(AllQuotes, '/quotes')
api.register_resource(SingleQuote, '/quote/<int:id>')

If you've been following along in vim or PyCharm or whatever, you'll notice we've registered SingleQuote at two URIs now:

    * `/quote` for getting a random quote
    * `/quote/<int:id>/` for getting a specific quote (or a random quote if it's out of range)

Flask-Restful resources can be registered on multiple endpoints. This isn't something special to these, every Flask URI resource can be registered multiple times. However, I also find having to do that registering by hand annoying. What if I needed to register the same resource in five places?

##Step Four: Creating our own Resource Type

The really nice thing about Flask-Restful is that we can take their implementations and shape them to our needs. For example, right now registering our resources look like this:

    api.register_resource(SingleQuote, '/quote')
    api.register_resource(SingleQuote, '/quote/<int:id>'
    api.register_resource(AllQuotes, '/quotes')

But let's say we wanted to put a random quote at `/` and make `/quote/all` an alias for `/quotes`? I think we can abstract away needing to retype so much stuff...

In [27]:
from flask.ext.restful import Resource as ResourceBase

class Resource(ResourceBase):
    routes = ()
    route_options = {}
    
    @classmethod
    def register(cls, api):
        api.register_resource(cls, *cls.routes, **cls.route_options)

By subclassing *our* resource instead of Flask-Restful's we can have our registration look like this instead:

In [ ]:
class SingleQuote(Resource):
    routes = ('/', '/quote', '/quote/<int:id>')
    
    def get(self, id=None):
        # as above, so below
        pass
    
    def post(self):
        # as above, so below
        pass

class AllQuotes(Resource):
    routes = ('/quotes', '/quote/all')
    
    def get(self):
        # as above, so below
        pass

SingleQuote.register(api)
AllQuotes.register(api)

Now, whether or not this is good practice or not is a matter of debate. I've heard strong arguments on both sides. Personally, I find being able to handle my resource registration like this is very clean and elegant. If I need more fine grained registration (say setting individual options at each end point), the `api.register_resource` is still available. Subclassing resource will also open another powerful tool which I'll explore in a bit.

##Step Five: Persistence

Right now all of our quotes are transient and only live in memory. It'd be nice to have some sort of persistence for our quotes. So, let's import Flask-SQLAlchemy and use that to handle the quotes instead.

In [ ]:
from flask.ext.sqlalchemy import SQLAlchemy
from sqlalchemy import func

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///quotes.db'
db = SQLAlchemy(app)

class Quote(db.Model):
    __tablename__ = 'quotes'
    
    id = db.Column(db.Integer, primary_key=True)
    # Unicode and UnicodeText over String and Text true Web 2.0 compatibility
    person = db.Column(db.Unicode(64)) 
    quote = db.Column(db.UnicodeText)
    
    def __init__(self, person, quote):
        self.person = person
        self.quote = quote
    
    # easy access to random quotes
    @classmethod
    def random(cls):
        return cls.query.order_by(func.random()).limit(1).first()
    
    # get quote or return a function call
    # example: Quote.get_or_call(quote_id, Quote.random)
    @classmethod
    def get_or_call(cls, id, func, *args, **kwargs):
        item = cls.query.get(id)
        if item is None:
            item = func(*args, **kwargs)
        return item

if __name__ == '__main__':
    db.create_all()
    
    quotes = [Quote(p, q) for p, q in [
            ('Herbert West', 'I was busy pushing bodies around as you well know '
             'and what would a note say, Dan? "Cat dead, details later"?'),
            ('Jack Burton', "You remember what ol' Jack Burton always says at a time like that: "
             '"Have ya paid your dues, Jack?" "Yessir, the check is in the mail."'),
            ('Igor', 'Well, why isn\'t it "Froaderick Fronkensteen"?')
    ]]

    db.session.add_all(quotes)
    db.session.commit()
    app.run(debug=True)

What's great about this is we don't need to change how QuoteSchema creates objects. The interface is still the same, but instead of building a namedtuple, we're now building a DAO with SQLAlchemy instead. Marshmallow doesn't know and doesn't care. We do, however, need to modify our resources:

In [ ]:
class SingleQuote(Resource):
    routes = ('/', '/quote', '/quote/<int:id>')
    
    def get(self, id=None):
        if id is None:
            q = Quote.random()
        else:
            q = Quote.get_or_call(id, Quote.random)
        return QuoteSchema().dump(q).data
    
    def post(self):
        """Add a quote"""
        json = request.get_json()
        if not json:
            return {'success': False, 'error': 'malformed request'} # probably should do something better
        
        else:
            try:
                if 'quotes' in json:
                    data = QuoteSchema().load(json['quotes'], many=True).data
                    db.session.add_all(data)
                    num = len(data)
                else:
                    data = QuoteSchema().load(json['quote']).data
                    db.session.add(data)
                    num = 1
                db.session.commit()
                return {'success': True, 'added': num}
            except KeyError as e: # super robust
                return {'success': False, 'error': repr(e)}

class AllQuotes(Resource):
    routes = ('/quotes', '/quote/all')
    
    def get(self):
        return QuoteSchema(many=True).dump(Quote.query.all()).data

There is one more bit of repetition I'd like to get rid of: Having `QuoteSchema(...).whatever` all over the place. Remember how I said I'd explore the power subclassing `flask_restful.Resource` gives us?

##Step Six: Our own automatic serialization

Instead of relying on Flask-Restful's automatic JSONification, we can instead say, "We're always going to use JSON (XML isn't webscale) so we'll provide our own serialization." Which is exactly what we're going to do:

In [28]:
from werkzeug.wrappers import Response as ResponseBase

class Resource(ResourceBase):
    schema = None
    routes = []
    route_options = {}
    
    @classmethod
    def register(cls, api):
        api.register_resource(cls, *cls.routes, **cls.route_options)
    
    def dispatch_method(self, *args, **kwargs):
        meth = getattr(self, request.method.lower(), None)
        if meth is None and request.method == 'HEAD':
            meth = getattr(self, 'get', None)
        assert meth is not None, "Unimplemented method {}".format(request.method)
        
        for decorator in self.method_decorators:
            meth = decorator(meth)
        
        resp = meth(*args, **kwargs)
        
        if isinstance(resp, ResponseBase):
            return resp
        elif self.schema:
            return jsonify(self.schema.dump(resp).data)
        else:
            return jsonify(resp)

Implementing our own `dispatch_method` allows us to override the standard Flask-Restful and flask.MethodView behaviors and handle JSON serialization on our own terms. This buys us the ability to provide a predefined schema instead of having to specify it in all the methods. We simple return a tuple of a dictionary of the data to JSONify and a boolean telling `dispatch_method` if it should use the schema or not (helpful for reporting non-serialized data). We'll need to modify our resources one last time (I promise). Sadly, without some work around, we can't return an arbitrary dictionary anymore, we need to return a fully realized Flask response (hence the `return jsonify({...})` below):

In [ ]:
class SingleQuote(Resource):
    schema = QuoteSchema()
    routes = ('/', '/quote', '/quote/<int:id>')

    def get(self, id=None):
        if id is None:
            return Quote.random()
        else:
            return Quote.get_or_call(id, Quote.random)

    def post(self):
        if request.mimetype != 'application/json':
            return jsonify({'badresult': repr(request.mimetype)})
        json = request.get_json()
        try:
            if 'quotes' in json:
                data = self.schema.load(json['quotes'], many=True).data
                db.session.add_all(data)
                num = len(data)
            else:
                data = self.schema.load(json['quote']).data
                db.session.add(data)
                num = 1
            db.session.commit()
            return jsonify({'result': True, 'added': num})
        except KeyError as e:
            return jsonify({'result': repr(e)})


class AllQuotes(Resource):
    schema = QuoteSchema(many=True)
    routes = ('/quotes', '/quote/all')

    def get(self):
        return Quote.query.all()
    
SingleQuote.register(api)
AllQuotes.register(api)

##That's all she wrote

Hopefully, this was a decent introduction to building a simple restful API with Flask and some of its extensions. There's more that can be done, such as not eliminating Flask-Restful's mimetype detection and automatic serialization of data for use, thus regaining the ability to return XML, CSV and other text types to the caller (of course, these have to implemented on your own, Flask-Restful only provides the mechanism for allowing it).

I've also glossed over quite a bit for space reasons, including completely ignoring one of my favorite Flask-related things: application factories, which are an amazingly powerful abstraction.

The full Quotr "application" is below:

In [ ]:
# Full Application to keep direction in mind:
from flask import request, jsonify, Flask
from flask.ext.restful import Api, Resource as ResourceBase
from flask.ext.sqlalchemy import SQLAlchemy
from marshmallow import Schema, post_dump
from sqlalchemy import func
from werkzeug.wrappers import Response as ResponseBase


app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///quotes.db'
api = Api(app)
db = SQLAlchemy(app)


class Quote(db.Model):
    __tablename__ = 'quotes'
    id = db.Column(db.Integer, primary_key=True)
    person = db.Column(db.String(64))
    quote = db.Column(db.Text)

    def __init__(self, person, quote):
        self.person = person
        self.quote = quote

    @classmethod
    def random(cls):
        return cls.query.order_by(func.random()).limit(1).first()

    @classmethod
    def get_or_call(cls, id, func, *args, **kwargs):
        data = cls.query.get(id)
        if data is None:
            data = func(*args, **kwargs)
        return data


class QuoteSchema(Schema):
    class Meta:
        additional = ('id', 'person', 'quote')

    @post_dump(raw=True)
    def add_root(self, data, many):
        name = self.__class__.__name__.lower().split('schema')[0]
        ns = name + 's' if many else name
        return {ns: data}
        
    def make_object(self, data):
        return Quote(person=data['person'], quote=data['quote'])

    
class Resource(ResourceBase):
    schema = None
    routes = []
    route_options = {}
    
    @classmethod
    def register(cls, api):
        api.register_resource(cls, *cls.routes, **cls.route_options)
    
    def dispatch_method(self, *args, **kwargs):
        meth = getattr(self, request.method.lower(), None)
        if meth is None and request.method == 'HEAD':
            meth = getattr(self, 'get', None)
        assert meth is not None, "Unimplemented method {}".format(request.method)
        
        for decorator in self.method_decorators:
            meth = decorator(meth)
        
        resp = meth(*args, **kwargs)
        
        if isinstance(resp, ResponseBase):
            return resp
        elif self.schema:
            return jsonify(self.schema.dump(resp).data)
        else:
            return jsonify(resp)
    

class SingleQuote(Resource):
    schema = QuoteSchema()
    routes = ('/', '/quote', '/quote/<int:id>')

    def get(self, id=None):
        if id is None:
            return Quote.random()
        else:
            return Quote.get_or_call(id, Quote.random)

    def post(self):
        if request.mimetype != 'application/json':
            return jsonify({'badresult': repr(request.mimetype)})
        json = request.get_json()
        try:
            if 'quotes' in json:
                data = self.schema.load(json['quotes'], many=True).data
                db.session.add_all(data)
                num = len(data)
            else:
                data = self.schema.load(json['quote']).data
                db.session.add(data)
                num = 1
            db.session.commit()
            return jsonify({'result': True, 'added': num})
        except KeyError as e:
            return jsonify({'result': repr(e)})


class AllQuotes(Resource):
    schema = QuoteSchema(many=True)
    routes = ('/quotes', '/quote/all')

    def get(self):
        return Quote.query.all()


SingleQuote.register(api)
AllQuotes.register(api)


if __name__ == '__main__':
    db.drop_all()   # remove for persistence of all quotes between sessions
    db.create_all() # remove after tables have been created
    db.session.commit()

    quotes = [Quote(p, q) for p, q in [
        ('Herbert West', 'I was busy pushing bodies around as you well know '
         'and what would a note say, Dan? "Cat dead, details later"?'),
        ('Jack Burton', "You remember what ol' Jack Burton always says at a "
         'time like that: "Have ya paid your dues, Jack?" "Yessir, the check '
         'is in the mail."'),
        ('Igor', 'Well, why isn\'t it "Froaderick Fronkensteen"?')
    ]]

    db.session.add_all(quotes)
    db.session.commit()
    app.run(debug=True)